# Looking into food price inflation using data from Tesco
> "Measuring food price inflation in a way that anyone in the UK can relate to."

- toc: true 
- badges: true
- comments: true
- author: Brandon Lee
- categories: [inflation, analysis]
- image: images/inflation_post.jpg

![](images/inflation_post.jpg "Source: https://www.rte.ie/news/business/2022/0908/1321302-cso-inflation-figures/")

**Disclaimer: The use of data collected for this study is covered by the Intellectual Property Office’s Exceptions to Copyright for Non-Commercial Research and Private Study (https://www.gov.uk/guidance/exceptions-to-copyright). No personal information has been collected as part of this study's data collection process and web scraping best practices implemented to the best of the author's abilities. The collected data will be inaccessible to the public to abide by Section 7 of the Terms and Conditions agreement laid out by Tesco (https://www.tesco.com/help/terms-and-conditions/).**

Since July 2021 the Consumer Prices Index (CPI) inflation rate in the UK has broken away from the Bank of England's target rate of 2%, steadily rising month by month. As of 3rd September 2022, the inflation rate stands at 10.1% with no indication of decreasing. What an awfully depressing way to start a blog post but unfortunately this is the reality that we are all currently facing in the UK. Every month the Office for National Statistics (ONS) calculates CPI by collecting the prices for a basket of around 730 different consumer goods and services that is a representation of the society's current buying habits and measures the average change in price of the basket on a year-over-year basis. At a risk of grossly simplifing, CPI tells us how much prices in general have changed over the last 12 months.

![](images/Figure_1_Annual_CPIH.png "Consumer Prices Index (CPI), Consumer Prices Index with Housing (CPIH), and Owner Occupier's Housing cost (OOH) component.")

That's all nice and well but society's buying habits doesn't necessarily reflect my buying habits. There are 12 broad categories of goods and services in the CPI basket, some of these categories such as clothing and footware, and furniture and household goods don't apply to me as I rarely ever buy new clothes or furniture. The full list of categories and goods and services of each category can be found on the [ONS website here](https://www.ons.gov.uk/economy/inflationandpriceindices/datasets/consumerpriceinflationbasketofgoodsandservices). One category that does apply to me, and to everyone else, is food and drinks. Food and drink goods (excluding alcohol) currently account for 9.3% of the weighting in CPI, which is a relatively small slice of the pie.

During recent times I've noticed that my shopping bill has gone up overall but I can't quite pin down what items are contributing the most to the this increase. To this end, I've built a cloud-based webscraping application that scrapes product data from Tesco's website including price and categories ([Github link](https://github.com/Blee1077/tesco-product-scrape) - blog article incoming). The reason why I chose Tesco as opposed to any other supermarket is because it's 5 minutes down the road from me.

In this article we'll load, process, transform, and analyse this scraped data to see how the prices of food categories have changed over time. If you want to skip directly to the charts and analysis, head to the 'Charts and Analysis' section in the above table of contents.

In [1]:
# hide_input

# Used to enable plotly
import plotly.io as pio
pio.renderers.default = 'notebook_connected'
from IPython.display import display, HTML
js = '<script src="https://cdnjs.cloudflare.com/ajax/libs/require.js/2.3.6/require.min.js" integrity="sha512-c3Nl8+7g4LMSTdrm621y7kf9v3SDPnhxLNhcjFJbKECVnmZHTdo+IRO05sNLTH/D3vA6u1X32ehoLC7WFVdheg==" crossorigin="anonymous"></script>'
display(HTML(js))

In [2]:
# hide
import boto3
import os
import io
import json
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly_express as px
import altair as alt
from functools import reduce
from IPython.display import HTML

pd.set_option('display.max_rows', 500)
s3_client = boto3.client('s3')

In [3]:
# hide
raw_df = pd.read_parquet(f's3://tesco-scrape-pipeline-app-tescoscrapes3bucket-1x8yycv16azsa/master_data/scraped_product_data.parquet')

In [4]:
# hide
dtype_map = {
    'name': str,
    'price_per_unit': float,
    'price_per_weight_quant': float,
    'weight_quant_unit': str,
    'offer': str,
    'category_1': str,
    'category_2': str,
    'category_3': str,
    'category_4': str,
    'clubcard_price_per_unit': float,
    'clubcard_discount_perc': float,
    'date': 'date'
}

# Drop product ID column and rows with NaN values in 'price_per_unit' column
preproc_df = (
    raw_df
    .drop(columns=['id'])
    .dropna(subset=['price_per_unit'])
)

# Convert columns to appropriate dtype
for col, dtype in dtype_map.items():
    if dtype == 'date':
        preproc_df[col] = pd.to_datetime(preproc_df[col], dayfirst=True)
    else:
        preproc_df[col] = preproc_df[col].astype(dtype)
        if dtype == str:
            preproc_df[col] = preproc_df[col].replace('nan', np.nan)

# Remove irrelevant categories and remove data past 2022-06-13 because it's low quality (many NaN values in columns)
cat1_mask = preproc_df['category_1'].isin(['baby', 'health-and-beauty', 'home-and-ents', 'summer-bbq', 'easter'])
date_mask = preproc_df['date'] <= '2022-06-13'
preproc_df = preproc_df[(~cat1_mask) & (date_mask)]

In [5]:
# hide
# Still some NaNs in category columns - about 4%
preproc_df.isna().mean()

name                       0.000000
price_per_unit             0.000000
price_per_weight_quant     0.000074
weight_quant_unit          0.000124
offer                      0.808033
category_1                 0.010347
category_2                 0.010347
category_3                 0.010347
category_4                 0.010347
clubcard_price_per_unit    0.898618
clubcard_discount_perc     0.898618
date                       0.000000
dtype: float64

In [6]:
# hide
# Shouldn't be any NaNs in the category columns if there are no NaNs in name column
# Remedy this by creating mappings for product name to product sub-category for all sub-categories and apply to name column

for sub_cat in ['category_1', 'category_2', 'category_3', 'category_4']:
    # Create mapping for product name to category sub-type
    prod_cat_map = {prod_name: cat for prod_name, cat in preproc_df[['name', sub_cat]].dropna().to_numpy()}
    
    # Apply mapping
    preproc_df[sub_cat] = preproc_df['name'].map(prod_cat_map)

In [7]:
# hide
# Much better now
preproc_df.isna().mean()

name                       0.000000
price_per_unit             0.000000
price_per_weight_quant     0.000074
weight_quant_unit          0.000124
offer                      0.808033
category_1                 0.001082
category_2                 0.001082
category_3                 0.001082
category_4                 0.001082
clubcard_price_per_unit    0.898618
clubcard_discount_perc     0.898618
date                       0.000000
dtype: float64

# Data Overview and Processing

The webscraping pipeline was ran on a near daily basis from 28th February 2022 to 13th June 2022, giving me 90 days worth of data. For each day, the names, price per unit and price per weight or quantity, product category and subcategories, promotional offers, and clubcard prices were collected. All prices are in pound sterling (£). The way that Tesco categorises products is through a primary (or parent) category followed by 3 subcategories, each more granular than the last.

The scraped data from Tesco is in the following format:

In [8]:
#  hide_input
reorder_cols = [
    'name','date','category_1','category_2','category_3','category_4',
    'price_per_unit','price_per_weight_quant','weight_quant_unit',
    'clubcard_price_per_unit','clubcard_discount_perc','offer'
]
rename_cols = {
    'name': 'Product Name',
    'category_1': 'Primary Category',
    'category_2': 'Secondary Category',
    'category_3': 'Tertiary Category',
    'category_4': 'Quaternary Category',
    'date': 'Date',
    'price_per_unit': 'Price per Unit (£)',
    'price_per_weight_quant': 'Price per Weight or Quantity (£)',
    'weight_quant_unit': 'Weight or Quantity Unit',
    'offer': 'Promotional Offer',
    'clubcard_price_per_unit': 'Clubcard Price per Unit (£)',
    'clubcard_discount_perc': 'Clubcard Discount (%)',
}

(
    preproc_df
    .head(3)
    .reset_index(drop=True)
    [reorder_cols]
    .rename(columns=rename_cols)
    .fillna(value='None')
)

,Product Name,Date,Primary Category,Secondary Category,Tertiary Category,Quaternary Category,Price per Unit (£),Price per Weight or Quantity (£),Weight or Quantity Unit,Clubcard Price per Unit (£),Clubcard Discount (%),Promotional Offer
0,Tesco Macaroni Pasta 1Kg,2022-02-28,food-cupboard,dried-pasta-rice-noodles-and-cous-cous,pasta-and-spaghetti,pasta-macaroni-penne-and-rigatoni,1.3,1.30,kg,None,None,None
1,Canderel 5 Refill Sachets 500Pk,2022-02-28,food-cupboard,home-baking-and-sugar,sweeteners-and-sugar-substitutes,sweetener-tablets,5.1,120.00,kg,None,None,None
2,Highland Still Spring Water 12 X 330Ml,2022-02-28,drinks,water,still-water,small-still-water,2.5,0.06,100ml,None,None,None


In its current state, the data is too granular to work with. It's reasonable to assume that the price of vast majority of products don't change on a day to day basis, so we'll transform the data into a weekly format where we'll take the median price of a product for the entire week commencing on Monday as the aggregation function. 

This will account for any pricing errors from Tesco's pricing department which could dramatically skew the mean value (e.g., accidentally setting the price of a bag of 5kg basmati rice as £500 instead of £5.00 for a day before being corrected will affect the weekly mean price but not the weekly median price). We'll also drop out columns that aren't needed for this piece of analysis. Namely the clubcard, promotional offer, and weight/quantity columns.

Once transformed, we'll take the example of the first 3 weeks of data for Tesco Ground Cinnamon (40G Jar) to see the format of the processed data:

In [9]:
# hide

# Transform price data to be on a weekly basis, use median as aggregation function
grouped_df = (
    preproc_df
    .set_index('date')
    .groupby(by=['name', 'category_1', 'category_2', 'category_3', 'category_4'])
    .resample('W-MON', label='left', closed='left')
    .agg({
        'price_per_unit': 'median',
    })
    .reset_index()
)
grouped_df['name'] = grouped_df['name'].str.strip()

In [10]:
# hide_input
(
    grouped_df
    [grouped_df['name'] == 'Tesco Ground Cinnamon 40G Jar']
    .head(3)
    .reset_index(drop=True)
    .rename(columns={
        'name': 'Product Name',
        'category_1': 'Primary Category',
        'category_2': 'Secondary Category',
        'category_3': 'Tertiary Category',
        'category_4': 'Quaternary Category',
        'date': 'Week Commencing Monday',
        'price_per_unit': 'Price per Unit (£)'
    })
    [['Product Name', 'Week Commencing Monday', 'Primary Category', 'Secondary Category','Tertiary Category','Quaternary Category','Price per Unit (£)']]
)

,Product Name,Week Commencing Monday,Primary Category,Secondary Category,Tertiary Category,Quaternary Category,Price per Unit (£)
0,Tesco Ground Cinnamon 40G Jar,2022-02-28,food-cupboard,cooking-ingredients,seasoning-herbs-and-spices,spices,0.85
1,Tesco Ground Cinnamon 40G Jar,2022-03-07,food-cupboard,cooking-ingredients,seasoning-herbs-and-spices,spices,0.85
2,Tesco Ground Cinnamon 40G Jar,2022-03-14,food-cupboard,cooking-ingredients,seasoning-herbs-and-spices,spices,0.85


Not much of a change when looking at 3 weeks worth of data for this product. While we're at it, let's take a look at how many different subcategories each main categories has:

In [11]:
# hide_input
row_lst = []
for cat in grouped_df['category_1'].unique():
    mask = grouped_df['category_1'] == cat
    
    tmp_lst = []
    for sub_cat_type in ['category_2', 'category_3', 'category_4']:
        tmp_lst.append(grouped_df[mask][sub_cat_type].nunique())
        
    row_lst.append(tmp_lst)

(
    pd.DataFrame(
        row_lst,
        columns=['category_2', 'category_3', 'category_4'],
        index=grouped_df['category_1'].unique()
    )
    .reset_index()
    .rename(
        columns={
            'index': 'Primary Category',
            'category_2': 'Number of Secondary Categories',
            'category_3': 'Number of Tertiary Categories',
            'category_4': 'Number of Quaternary Categories',
        }
    )
)

,Primary Category,Number of Secondary Categories,Number of Tertiary Categories,Number of Quaternary Categories
0,fresh-food,19,180,606
1,drinks,18,77,272
2,food-cupboard,18,173,682
3,bakery,10,52,78
4,frozen-food,15,76,148


It's very clear that fresh food and food cupboard main categories have the largest amount of products judging by the number of quaternary categories they have. The bakery main category seems to be the smallest with not much of an increase going from tertiarty to quaternary categories.

# Calculating Index Numbers

Before we can go on with our analysis, we need to first understand what index numbers are and why they're used, and how we can use it in the context of this analysis piece. 

In essence, index numbers are used to measure changes and simplify comparisons. In particular, they're used in the field of economics to measure changes in price (i.e., inflation or deflation) and changes in quantity (e.g., sales volumes or economic output). The way that index numbers are calculated is to take a base value from a particular point in time and determine the ratio between the base value and a value from a future point in time. 

As an example, imagine we want to understand how much the price of a standard Oreo McFlurry from McDonalds changes over time. I'd go into my local store one day and note the price down, let's say it's £0.80. I come back 2 months later and the price has gone up to £1.00. To calculate the index we would need to divide the new price by the base price and multiply the result by 100, which would give us 125. We interpret this as a 25% increase in price over the 2 month time period. Of course I could go in another 2 months later, note the price, and calculate another index number using the base price to see how the price has changed over a 4 month time period.

This line of thinking can be naturally extended to consider baskets of products instead of a singluar product by taking the sum of the prices of products in the basket.

In fact, that's exactly what we'll do. For each of the primary, secondary, tertiary, and quaternary categories we'll calculate the total value of the basket of products and use that to calculate the index numbers. But hang on a moment here, what if Tesco decides to de-list a product and stop selling it - what happens then? It'll mean that the total basket value is no longer comparable between different points in time, meaning that the index numbers are misleading. To combat this, I've only taken products within categories that are present across **all of the weeks in the dataset** so that the total value of a category's basket is comparable across time. any subcategories that have less than 5 items are excluded to save on processing time.

At the end of this processing step we'll have 4 datasets, one for each level of category granularity:

In [12]:
# hide
categories = ['category_1', 'category_2', 'category_3', 'category_4']
cat_index_dict = {}
skipped_cat_dict= {}

for sub_cat_type in categories:
    
    tmp_df = (
        grouped_df
        .dropna(subset=['price_per_unit'])
        .groupby(by=['date']+categories[:(int(sub_cat_type.split('_')[-1]))])
        ['name']
        .unique()
        .reset_index()
    )

    master_index_df = pd.DataFrame()
    prod_prop_dict = {}

    for cat in tmp_df[sub_cat_type].unique():
        cat_mask = tmp_df[sub_cat_type] == cat
        
        prod_lst = reduce(lambda left, right: set(left).intersection(right), tmp_df[cat_mask]['name'])
        union_prod_lst = reduce(lambda left, right: set(left).union(right), tmp_df[cat_mask]['name'])
        prod_prop_dict[cat] = len(prod_lst) / len(union_prod_lst)
        
        if len(prod_lst) < 5:
            if sub_cat_type not in skipped_cat_dict:
                skipped_cat_dict[sub_cat_type] = [cat]
            else:
                skipped_cat_dict[sub_cat_type].append(cat)
            continue

        prod_mask = grouped_df['name'].isin(prod_lst)

        index_df = (
            grouped_df
            [prod_mask]
            .groupby(by=['date']+categories[:(int(sub_cat_type.split('_')[-1]))])
            ['price_per_unit']
            .sum()
            .reset_index()
            .rename(columns={'price_per_unit': 'total_basket_value'})
        )
        index_df['index'] = 100 * (index_df['total_basket_value'] / index_df['total_basket_value'].iloc[0])
        
        master_index_df = pd.concat([master_index_df, index_df])
        
    cat_index_dict[sub_cat_type] = master_index_df

In [13]:
# hide_input
mask = cat_index_dict['category_1']['category_1']=='fresh-food'
(
    cat_index_dict['category_1']
    [mask]
    .head(3)
    .rename(columns={
        'date': 'Week Commencing Monday',
        'category_1': 'Primary Category',
        'total_basket_value': 'Total Basket Value',
        'index': 'Index'
    })
)

,Week Commencing Monday,Primary Category,Total Basket Value,Index
0,2022-02-28,fresh-food,6298.790,100.000000
1,2022-03-07,fresh-food,6306.640,100.124627
2,2022-03-14,fresh-food,6311.605,100.203452


In [14]:
# hide_input
mask = (
    (cat_index_dict['category_2']['category_1']=='fresh-food')
    & (cat_index_dict['category_2']['category_2']=='fresh-meat-and-poultry')
)

(
    cat_index_dict['category_2']
    [mask]
    .head(3)
    .rename(columns={
        'date': 'Week Commencing Monday',
        'category_1': 'Primary Category',
        'category_2': 'Secondary Category',
        'total_basket_value': 'Total Basket Value',
        'index': 'Index'
    })
)

,Week Commencing Monday,Primary Category,Secondary Category,Total Basket Value,Index
0,2022-02-28,fresh-food,fresh-meat-and-poultry,1252.15,100.000000
1,2022-03-07,fresh-food,fresh-meat-and-poultry,1252.32,100.013577
2,2022-03-14,fresh-food,fresh-meat-and-poultry,1254.22,100.165316


In [15]:
# hide_input
mask = (
    (cat_index_dict['category_3']['category_1']=='fresh-food')
    & (cat_index_dict['category_3']['category_2']=='fresh-meat-and-poultry')
    & (cat_index_dict['category_3']['category_3']=='fresh-chicken')
)

(
    cat_index_dict['category_3']
    [mask]
    .head(3)
    .rename(columns={
        'date': 'Week Commencing Monday',
        'category_1': 'Primary Category',
        'category_2': 'Secondary Category',
        'category_3': 'Tertiary Category',
        'total_basket_value': 'Total Basket Value',
        'index': 'Index'
    })
)

,Week Commencing Monday,Primary Category,Secondary Category,Tertiary Category,Total Basket Value,Index
0,2022-02-28,fresh-food,fresh-meat-and-poultry,fresh-chicken,172.34,100.000000
1,2022-03-07,fresh-food,fresh-meat-and-poultry,fresh-chicken,172.51,100.098642
2,2022-03-14,fresh-food,fresh-meat-and-poultry,fresh-chicken,172.51,100.098642


In [16]:
# hide_input
mask = (
    (cat_index_dict['category_4']['category_1']=='fresh-food')
    & (cat_index_dict['category_4']['category_2']=='fresh-meat-and-poultry')
    & (cat_index_dict['category_4']['category_3']=='fresh-chicken')
    & (cat_index_dict['category_4']['category_4']=='chicken-breast')
)

(
    cat_index_dict['category_4']
    [mask]
    .head(3)
    .rename(columns={
        'date': 'Week Commencing Monday',
        'category_1': 'Primary Category',
        'category_2': 'Secondary Category',
        'category_3': 'Tertiary Category',
        'category_4': 'Quaternary Category',
        'total_basket_value': 'Total Basket Value',
        'index': 'Index'
    })
)

,Week Commencing Monday,Primary Category,Secondary Category,Tertiary Category,Quaternary Category,Total Basket Value,Index
0,2022-02-28,fresh-food,fresh-meat-and-poultry,fresh-chicken,chicken-breast,42.53,100.0
1,2022-03-07,fresh-food,fresh-meat-and-poultry,fresh-chicken,chicken-breast,42.53,100.0
2,2022-03-14,fresh-food,fresh-meat-and-poultry,fresh-chicken,chicken-breast,42.53,100.0


In [17]:
# hide
for cat in skipped_cat_dict:
    print(cat, len(skipped_cat_dict[cat]))

category_2 4
category_3 145
category_4 1036


# Charts and Analysis

## Overview of Price Changes for All Categories

Let's start with the big picture (quite literally), how did prices change when comparing the basket values of categories?

Note that the **index numbers presented in the graph below are accurate when rounding to whole numbers**. 

Technical explanation - The numbers get more inaccurate as you go up category granularities due to the way that the index numbers are aggregated. I'm using the most granular dataset that contains index numbers of quaternary categories, these are then aggregated up to the tertiary category level using the harmonic mean weighted by the basket value of the quarternary categories within the tertiary category. This process repeats for the secondary and then the primary categories, the approximation error increasing going up with each category level.

In [29]:
# hide

# Calculate overall price increase between beginning and end period
earliest_date_mask = cat_index_dict['category_1']['date'] == '2022-02-28'
latest_date_mask = cat_index_dict['category_1']['date'] == '2022-06-13'

(100 * (
    cat_index_dict['category_1'][latest_date_mask]['total_basket_value'].sum() /
    cat_index_dict['category_1'][earliest_date_mask]['total_basket_value'].sum()
    )
)

101.70219405149523

**Tip: This chart is interactive!** \
Hover your mouse over the sector to look at the index number of categories (if on mobile, tap and hold the name of the category). Clicking on a sector will focus on that particular category and its subcategories. There's 4 levels of categories but only 3 levels are shown at any one time for better performance.

In [18]:
# hide

# Plotly Express' Sunburst chart aggregates the column given as color argument using weighted arithmetic mean
# Since we want to use index number column as color argument, weighted arithmentic mean isn't appropriate to use
# It would be better to use the weighted harmonic mean as index numbers are rates but plotly doesn't given an option to use that as an argument
# So I'll need to take bits and pieces of their code just to change one line in one function...
import math
import scipy.stats as scp
import plotly.graph_objects as go
from collections import namedtuple, OrderedDict
from plotly.express._special_inputs import IdentityMap, Constant, Range
from plotly.express._core import process_dataframe_pie, process_dataframe_timeline, _is_continuous, \
                                configure_animation_controls, configure_axes, _set_trace_grid_reference, init_figure, apply_default_cascade, \
                                build_dataframe, make_trace_kwargs, ColorscaleValidator, get_decorated_label, _subplot_type_for_trace_type, \
                                infer_config, one_group, get_groups_and_orders, get_label


def sunburst(
    data_frame=None,
    names=None,
    values=None,
    parents=None,
    path=None,
    ids=None,
    color=None,
    color_continuous_scale=None,
    range_color=None,
    color_continuous_midpoint=None,
    color_discrete_sequence=None,
    color_discrete_map=None,
    hover_name=None,
    hover_data=None,
    custom_data=None,
    labels=None,
    title=None,
    template=None,
    width=None,
    height=None,
    branchvalues=None,
    maxdepth=None,
) -> go.Figure:
    """
    A sunburst plot represents hierarchial data as sectors laid out over
    several levels of concentric rings.
    """
    if color_discrete_sequence is not None:
        layout_patch = {"sunburstcolorway": color_discrete_sequence}
    else:
        layout_patch = {}
    if path is not None and (ids is not None or parents is not None):
        raise ValueError(
            "Either `path` should be provided, or `ids` and `parents`."
            "These parameters are mutually exclusive and cannot be passed together."
        )
    if path is not None and branchvalues is None:
        branchvalues = "total"
    return make_figure(
        args=locals(),
        constructor=go.Sunburst,
        trace_patch=dict(branchvalues=branchvalues, maxdepth=maxdepth),
        layout_patch=layout_patch,
    )


def make_figure(args, constructor, trace_patch=None, layout_patch=None):
    trace_patch = trace_patch or {}
    layout_patch = layout_patch or {}
    apply_default_cascade(args)

    args = build_dataframe(args, constructor)
    if constructor in [go.Treemap, go.Sunburst, go.Icicle] and args["path"] is not None:
        args = process_dataframe_hierarchy(args)
    if constructor in [go.Pie]:
        args, trace_patch = process_dataframe_pie(args, trace_patch)
    if constructor == "timeline":
        constructor = go.Bar
        args = process_dataframe_timeline(args)

    trace_specs, grouped_mappings, sizeref, show_colorbar = infer_config(
        args, constructor, trace_patch, layout_patch
    )
    grouper = [x.grouper or one_group for x in grouped_mappings] or [one_group]
    groups, orders = get_groups_and_orders(args, grouper)

    col_labels = []
    row_labels = []
    nrows = ncols = 1
    for m in grouped_mappings:
        if m.grouper not in orders:
            m.val_map[""] = m.sequence[0]
        else:
            sorted_values = orders[m.grouper]
            if m.facet == "col":
                prefix = get_label(args, args["facet_col"]) + "="
                col_labels = [prefix + str(s) for s in sorted_values]
                ncols = len(col_labels)
            if m.facet == "row":
                prefix = get_label(args, args["facet_row"]) + "="
                row_labels = [prefix + str(s) for s in sorted_values]
                nrows = len(row_labels)
            for val in sorted_values:
                if val not in m.val_map:  # always False if it's an IdentityMap
                    m.val_map[val] = m.sequence[len(m.val_map) % len(m.sequence)]

    subplot_type = _subplot_type_for_trace_type(constructor().type)

    trace_names_by_frame = {}
    frames = OrderedDict()
    trendline_rows = []
    trace_name_labels = None
    facet_col_wrap = args.get("facet_col_wrap", 0)
    for group_name, group in groups.items():
        mapping_labels = OrderedDict()
        trace_name_labels = OrderedDict()
        frame_name = ""
        for col, val, m in zip(grouper, group_name, grouped_mappings):
            if col != one_group:
                key = get_label(args, col)
                if not isinstance(m.val_map, IdentityMap):
                    mapping_labels[key] = str(val)
                    if m.show_in_trace_name:
                        trace_name_labels[key] = str(val)
                if m.variable == "animation_frame":
                    frame_name = val
        trace_name = ", ".join(trace_name_labels.values())
        if frame_name not in trace_names_by_frame:
            trace_names_by_frame[frame_name] = set()
        trace_names = trace_names_by_frame[frame_name]

        for trace_spec in trace_specs:
            # Create the trace
            trace = trace_spec.constructor(name=trace_name)
            if trace_spec.constructor not in [
                go.Parcats,
                go.Parcoords,
                go.Choropleth,
                go.Choroplethmapbox,
                go.Densitymapbox,
                go.Histogram2d,
                go.Sunburst,
                go.Treemap,
                go.Icicle,
            ]:
                trace.update(
                    legendgroup=trace_name,
                    showlegend=(trace_name != "" and trace_name not in trace_names),
                )
            if trace_spec.constructor in [go.Bar, go.Violin, go.Box, go.Histogram]:
                trace.update(alignmentgroup=True, offsetgroup=trace_name)
            trace_names.add(trace_name)

            # Init subplot row/col
            trace._subplot_row = 1
            trace._subplot_col = 1

            for i, m in enumerate(grouped_mappings):
                val = group_name[i]
                try:
                    m.updater(trace, m.val_map[val])  # covers most cases
                except ValueError:
                    # this catches some odd cases like marginals
                    if (
                        trace_spec != trace_specs[0]
                        and (
                            trace_spec.constructor in [go.Violin, go.Box]
                            and m.variable in ["symbol", "pattern", "dash"]
                        )
                        or (
                            trace_spec.constructor in [go.Histogram]
                            and m.variable in ["symbol", "dash"]
                        )
                    ):
                        pass
                    elif (
                        trace_spec != trace_specs[0]
                        and trace_spec.constructor in [go.Histogram]
                        and m.variable == "color"
                    ):
                        trace.update(marker=dict(color=m.val_map[val]))
                    elif (
                        trace_spec.constructor in [go.Choropleth, go.Choroplethmapbox]
                        and m.variable == "color"
                    ):
                        trace.update(
                            z=[1] * len(group),
                            colorscale=[m.val_map[val]] * 2,
                            showscale=False,
                            showlegend=True,
                        )
                    else:
                        raise

                # Find row for trace, handling facet_row and marginal_x
                if m.facet == "row":
                    row = m.val_map[val]
                else:
                    if (
                        args.get("marginal_x") is not None  # there is a marginal
                        and trace_spec.marginal != "x"  # and we're not it
                    ):
                        row = 2
                    else:
                        row = 1

                # Find col for trace, handling facet_col and marginal_y
                if m.facet == "col":
                    col = m.val_map[val]
                    if facet_col_wrap:  # assumes no facet_row, no marginals
                        row = 1 + ((col - 1) // facet_col_wrap)
                        col = 1 + ((col - 1) % facet_col_wrap)
                else:
                    if trace_spec.marginal == "y":
                        col = 2
                    else:
                        col = 1

                if row > 1:
                    trace._subplot_row = row

                if col > 1:
                    trace._subplot_col = col
            if (
                trace_specs[0].constructor == go.Histogram2dContour
                and trace_spec.constructor == go.Box
                and trace.line.color
            ):
                trace.update(marker=dict(color=trace.line.color))

            if "ecdfmode" in args:
                base = args["x"] if args["orientation"] == "v" else args["y"]
                var = args["x"] if args["orientation"] == "h" else args["y"]
                ascending = args.get("ecdfmode", "standard") != "reversed"
                group = group.sort_values(by=base, ascending=ascending)
                group_sum = group[var].sum()  # compute here before next line mutates
                group[var] = group[var].cumsum()
                if not ascending:
                    group = group.sort_values(by=base, ascending=True)

                if args.get("ecdfmode", "standard") == "complementary":
                    group[var] = group_sum - group[var]

                if args["ecdfnorm"] == "probability":
                    group[var] = group[var] / group_sum
                elif args["ecdfnorm"] == "percent":
                    group[var] = 100.0 * group[var] / group_sum

            patch, fit_results = make_trace_kwargs(
                args, trace_spec, group, mapping_labels.copy(), sizeref
            )
            trace.update(patch)
            if fit_results is not None:
                trendline_rows.append(mapping_labels.copy())
                trendline_rows[-1]["px_fit_results"] = fit_results
            if frame_name not in frames:
                frames[frame_name] = dict(data=[], name=frame_name)
            frames[frame_name]["data"].append(trace)
    frame_list = [f for f in frames.values()]
    if len(frame_list) > 1:
        frame_list = sorted(
            frame_list, key=lambda f: orders[args["animation_frame"]].index(f["name"])
        )

    if show_colorbar:
        colorvar = "z" if constructor in [go.Histogram2d, go.Densitymapbox] else "color"
        range_color = args["range_color"] or [None, None]

        colorscale_validator = ColorscaleValidator("colorscale", "make_figure")
        layout_patch["coloraxis1"] = dict(
            colorscale=colorscale_validator.validate_coerce(
                args["color_continuous_scale"]
            ),
            cmid=args["color_continuous_midpoint"],
            cmin=range_color[0],
            cmax=range_color[1],
            colorbar=dict(
                title_text=get_decorated_label(args, args[colorvar], colorvar)
            ),
        )
    for v in ["height", "width"]:
        if args[v]:
            layout_patch[v] = args[v]
    layout_patch["legend"] = dict(tracegroupgap=0)
    if trace_name_labels:
        layout_patch["legend"]["title_text"] = ", ".join(trace_name_labels)
    if args["title"]:
        layout_patch["title_text"] = args["title"]
    elif args["template"].layout.margin.t is None:
        layout_patch["margin"] = {"t": 60}
    if (
        "size" in args
        and args["size"]
        and args["template"].layout.legend.itemsizing is None
    ):
        layout_patch["legend"]["itemsizing"] = "constant"

    if facet_col_wrap:
        nrows = math.ceil(ncols / facet_col_wrap)
        ncols = min(ncols, facet_col_wrap)

    if args.get("marginal_x") is not None:
        nrows += 1

    if args.get("marginal_y") is not None:
        ncols += 1

    fig = init_figure(
        args, subplot_type, frame_list, nrows, ncols, col_labels, row_labels
    )

    # Position traces in subplots
    for frame in frame_list:
        for trace in frame["data"]:
            if isinstance(trace, go.Splom):
                # Special case that is not compatible with make_subplots
                continue

            _set_trace_grid_reference(
                trace,
                fig.layout,
                fig._grid_ref,
                nrows - trace._subplot_row + 1,
                trace._subplot_col,
            )

    # Add traces, layout and frames to figure
    fig.add_traces(frame_list[0]["data"] if len(frame_list) > 0 else [])
    fig.update_layout(layout_patch)
    if "template" in args and args["template"] is not None:
        fig.update_layout(template=args["template"], overwrite=True)
    for f in frame_list:
        f["name"] = str(f["name"])
    fig.frames = frame_list if len(frames) > 1 else []

    if args.get("trendline") and args.get("trendline_scope", "trace") == "overall":
        trendline_spec = make_trendline_spec(args, constructor)
        trendline_trace = trendline_spec.constructor(
            name="Overall Trendline", legendgroup="Overall Trendline", showlegend=False
        )
        if "line" not in trendline_spec.trace_patch:  # no color override
            for m in grouped_mappings:
                if m.variable == "color":
                    next_color = m.sequence[len(m.val_map) % len(m.sequence)]
                    trendline_spec.trace_patch["line"] = dict(color=next_color)
        patch, fit_results = make_trace_kwargs(
            args, trendline_spec, args["data_frame"], {}, sizeref
        )
        trendline_trace.update(patch)
        fig.add_trace(
            trendline_trace, row="all", col="all", exclude_empty_subplots=True
        )
        fig.update_traces(selector=-1, showlegend=True)
        if fit_results is not None:
            trendline_rows.append(dict(px_fit_results=fit_results))

    fig._px_trendlines = pd.DataFrame(trendline_rows)

    configure_axes(args, constructor, fig, orders)
    configure_animation_controls(args, constructor, fig)
    return fig


def _check_dataframe_all_leaves(df):
    df_sorted = df.sort_values(by=list(df.columns))
    null_mask = df_sorted.isnull()
    df_sorted = df_sorted.astype(str)
    null_indices = np.nonzero(null_mask.any(axis=1).values)[0]
    for null_row_index in null_indices:
        row = null_mask.iloc[null_row_index]
        i = np.nonzero(row.values)[0][0]
        if not row[i:].all():
            raise ValueError(
                "None entries cannot have not-None children",
                df_sorted.iloc[null_row_index],
            )
    df_sorted[null_mask] = ""
    row_strings = list(df_sorted.apply(lambda x: "".join(x), axis=1))
    for i, row in enumerate(row_strings[:-1]):
        if row_strings[i + 1] in row and (i + 1) in null_indices:
            raise ValueError(
                "Non-leaves rows are not permitted in the dataframe \n",
                df_sorted.iloc[i + 1],
                "is not a leaf.",
            )


# The one line of code that needs to be changed is in the inner function named 'aggfunc_continuous'
def process_dataframe_hierarchy(args):
    """
    Build dataframe for sunburst, treemap, or icicle when the path argument is provided.
    """
    df = args["data_frame"]
    path = args["path"][::-1]
    _check_dataframe_all_leaves(df[path[::-1]])
    discrete_color = False

    new_path = []
    for col_name in path:
        new_col_name = col_name + "_path_copy"
        new_path.append(new_col_name)
        df[new_col_name] = df[col_name]
    path = new_path
    # ------------ Define aggregation functions --------------------------------

    def aggfunc_discrete(x):
        uniques = x.unique()
        if len(uniques) == 1:
            return uniques[0]
        else:
            return "(?)"

    agg_f = {}
    aggfunc_color = None
    if args["values"]:
        try:
            df[args["values"]] = pd.to_numeric(df[args["values"]])
        except ValueError:
            raise ValueError(
                "Column `%s` of `df` could not be converted to a numerical data type."
                % args["values"]
            )

        if args["color"]:
            if args["color"] == args["values"]:
                new_value_col_name = args["values"] + "_sum"
                df[new_value_col_name] = df[args["values"]]
                args["values"] = new_value_col_name
        count_colname = args["values"]
    else:
        # we need a count column for the first groupby and the weighted mean of color
        # trick to be sure the col name is unused: take the sum of existing names
        count_colname = (
            "count"
            if "count" not in df.columns
            else "".join([str(el) for el in list(df.columns)])
        )
        # we can modify df because it's a copy of the px argument
        df[count_colname] = 1
        args["values"] = count_colname
    agg_f[count_colname] = "sum"

    if args["color"]:
        if not _is_continuous(df, args["color"]):
            aggfunc_color = aggfunc_discrete
            discrete_color = True
        else:

            def aggfunc_continuous(x):
                return scp.hmean(x, weights=df.loc[x.index, count_colname])

            aggfunc_color = aggfunc_continuous
        agg_f[args["color"]] = aggfunc_color

    #  Other columns (for color, hover_data, custom_data etc.)
    cols = list(set(df.columns).difference(path))
    for col in cols:  # for hover_data, custom_data etc.
        if col not in agg_f:
            agg_f[col] = aggfunc_discrete
    # Avoid collisions with reserved names - columns in the path have been copied already
    cols = list(set(cols) - set(["labels", "parent", "id"]))
    # ----------------------------------------------------------------------------
    df_all_trees = pd.DataFrame(columns=["labels", "parent", "id"] + cols)
    #  Set column type here (useful for continuous vs discrete colorscale)
    for col in cols:
        df_all_trees[col] = df_all_trees[col].astype(df[col].dtype)
    for i, level in enumerate(path):
        df_tree = pd.DataFrame(columns=df_all_trees.columns)
        dfg = df.groupby(path[i:]).agg(agg_f)
        dfg = dfg.reset_index()
        # Path label massaging
        df_tree["labels"] = dfg[level].copy().astype(str)
        df_tree["parent"] = ""
        df_tree["id"] = dfg[level].copy().astype(str)
        if i < len(path) - 1:
            j = i + 1
            while j < len(path):
                df_tree["parent"] = (
                    dfg[path[j]].copy().astype(str) + "/" + df_tree["parent"]
                )
                df_tree["id"] = dfg[path[j]].copy().astype(str) + "/" + df_tree["id"]
                j += 1

        df_tree["parent"] = df_tree["parent"].str.rstrip("/")
        if cols:
            df_tree[cols] = dfg[cols]
        df_all_trees = pd.concat([df_all_trees, df_tree], ignore_index=True)

    # we want to make sure than (?) is the first color of the sequence
    if args["color"] and discrete_color:
        sort_col_name = "sort_color_if_discrete_color"
        while sort_col_name in df_all_trees.columns:
            sort_col_name += "0"
        df_all_trees[sort_col_name] = df[args["color"]].astype(str)
        df_all_trees = df_all_trees.sort_values(by=sort_col_name)

    # Now modify arguments
    args["data_frame"] = df_all_trees
    args["path"] = None
    args["ids"] = "id"
    args["names"] = "labels"
    args["parents"] = "parent"
    if args["color"]:
        if not args["hover_data"]:
            args["hover_data"] = [args["color"]]
        elif isinstance(args["hover_data"], dict):
            if not args["hover_data"].get(args["color"]):
                args["hover_data"][args["color"]] = (True, None)
        else:
            args["hover_data"].append(args["color"])
    return args

In [19]:
# hide_input

tmp_df = cat_index_dict['category_4'].reset_index(drop=True).copy()
latest_date_mask = tmp_df['date'] == '2022-06-13'

fig = sunburst(
    tmp_df[latest_date_mask].drop_duplicates(subset=['category_4']),
    path=['category_1', 'category_2', 'category_3', 'category_4'],
    values='total_basket_value',
    color='index',
    branchvalues='total',
    range_color=[90,110],
    color_continuous_scale='balance',
    color_continuous_midpoint=100,
    width=1100,
    height=950,
    maxdepth=3
)

fig.update_coloraxes(colorbar=dict(len=0.75))
fig.update_layout(margin=dict(l=0, r=0, t=20, b=20))
fig.show()

Straight away we can see that fresh food and frozen food have increased by ~3% and ~4% respectively. The major contributing subcategories for fresh food are cheese (~5% increase); fresh meat and poultry (~5% increase); ready meals (~4% increase); and yogurts (~4% increase). For frozen food, the major contributing subcategories are frozen fish and seafood (~9% increase); frozen chips, onion rings, potatoes, and rice (~12% increase); and frozen meat and poultry (~5% increase). 

Yikes, no wonder why my shopping bill has been going up. Fresh chicken breast and thighs have gone up by over 10%!

Overall, prices have by 1.7% across all categories and products in the processed dataset.

We can also see there are a few secondary categories that have gone down, most notably is the cooking sauces, meal kits, and sides subcategory within the food cupboard cateogry which has gone down by ~2%. This does raise another question though, how many categories in each level experienced an increase, decrease, or no change in price?

In [23]:
# hide
data_list = []
cat_type_map = {
    'category_2': 'Secondary Categories',
    'category_3': 'Tertiary Categories',
    'category_4': 'Quaternary Categories'
}
for cat_type in ['category_2', 'category_3', 'category_4']:
    date_mask = cat_index_dict[cat_type]['date'] == '2022-06-13'
    
    for cat_1 in cat_index_dict[cat_type]['category_1'].unique():
        cat_1_mask = cat_index_dict[cat_type]['category_1'] == cat_1
        mapped_cat_type = cat_type_map[cat_type]
        # cat_title = ' '.join(cat_1.split('-')).title()
        
        data_list.append({
            'Category Type': mapped_cat_type,
            'Primary Category': cat_1,
            'Number': (cat_index_dict[cat_type][date_mask & cat_1_mask]['index'] > 100).sum(),
            'Change': 'Increase'
        })
        
        data_list.append({
            'Category Type': mapped_cat_type,
            'Primary Category': cat_1,
            'Number': (cat_index_dict[cat_type][date_mask & cat_1_mask]['index'] < 100).sum(),
            'Change': 'Decrease'
        })
        
        data_list.append({
            'Category Type': mapped_cat_type,
            'Primary Category': cat_1,
            'Number': (cat_index_dict[cat_type][date_mask & cat_1_mask]['index'] == 100).sum(),
            'Change': 'No Change'
        })

changes_df = pd.DataFrame(data_list)
changes_df = changes_df.merge(
    right = (changes_df
    .groupby(by=['Category Type', 'Primary Category'])
    ['Number']
    .sum()
    .reset_index()
    .rename(columns={'Number': 'Sum'})),
    on=['Category Type', 'Primary Category'],
    how='left'
)
changes_df['Percentage'] = (changes_df['Number'] / changes_df['Sum'])

In [24]:
# hide_input
order = ['Decrease', 'No Change', 'Increase'][::-1]
range_ = ['#4c78a8', '#cccccc', '#e45756'][::-1]

alt.Chart(changes_df).transform_calculate(
    order=f"-indexof({order}, datum.Change)"
).mark_bar().encode(
    x=alt.X('Percentage', axis=alt.Axis(title="Percent", format="%")),
    y='Primary Category',
    color=alt.Color('Change', scale=alt.Scale(domain=order, range=range_)),
    column=alt.Column('Category Type', sort=['Secondary Categories', 'Tertiary Categories', 'Quaternary Categories']),
    order="order:Q",
    tooltip=[
        alt.Tooltip('Number', title="Number of Subcategories"),
        alt.Tooltip('Sum', title="Total Subcategories"),
        alt.Tooltip('Percentage', title="Percentage", format='.1%'),
    ],
).properties(
    width=270,
    height=250,
    title='Percentage of subcategories that have increased, decreased, or stayed the same in basket value',
).configure_title(
    fontSize=17,
    dy=-25,
).configure_header(
    titleFontSize=15,
    labelFontSize=14
).configure_axis(
    labelFontSize=14,
    titleFontSize=15
).configure_legend(
titleFontSize=14,
labelFontSize=13
) 

alt.Chart(...)

It's very clear from the above that the bakery and frozen food primary categories had all secondary, tertiary, and quaternary subcategories either increase in price or stay the same! We'll have to note that they're the two smallest primary categories by both total basket value and number of subcategories, but there are ongoing macroeconomic events that are affecting both the supply chain and cost of businesses. 

As of writing, the 2022 Russian invasion of Ukraine is affecting the two largest global suppliers of wheat. Russia produces 11% of the world's wheat and accounts for 19% of global wheat exports. On the other hand, Ukraine produces 3% of the world's wheat and accounts for 9% of global wheat exports. Both of these together account for more than a quarter of the world's wheat export market {% fn 1 %}. You can guess what ingredient bakery goods use that's derived from wheat.

Energy prices have also been rising throughout 2022 and are forecasted to reach unprecendented levels during the first two quarters of 2023. The eletricity price (kWh) cap rose from 20.8p in Q1 2022 to 28.3p in Q2 2022 {% fn 2 %}, an increase of 36%. Guess what's needed to keep frozen food frozen.

Generally speaking, it does seem that the vast majority of subcategories have experienced an increase in price, no matter what level of category granularity is being looked at. The drinks and food cupboard primary categories are the least affected as they're the only two that contain secondary categories that have decreased in price; that being said they only account for 11.1% and 18.8% of the total number of their respective total secondary categories.

{{ 'Source: https://asmith.ucdavis.edu/news/russia-ukraine' | fndetail: 1 }}
{{ 'Source: https://www.icaew.com/insights/viewpoints-on-the-news/2022/sept-2022/chart-of-the-week-energy-price-cap-update' | fndetail: 2 }}

## Price Changes for Primary Food Categories

Let's now take a look at how the index of primary categories changed over time as opposed to a snapshot in time.

In [22]:
# hide_input

selection = alt.selection_multi(fields=['category_1'], bind='legend')

base = alt.Chart(cat_index_dict['category_1'].reset_index()).mark_line().encode(
    y=alt.Y(shorthand='index', scale=alt.Scale(zero=False), axis=alt.Axis(title='Index')),
    x=alt.X(shorthand='date', axis=alt.Axis(title='Week Commencing Monday')),
    color=alt.Color('category_1', legend=alt.Legend(title="Food Category")),
    strokeDash='category_1',
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
).add_selection(
    selection
).properties(
    title='Index of Main Tesco Food Categories',
    width=900,
    height=500
)

hover = alt.selection_single(
    fields=["date"],
    nearest=True,
    on="mouseover",
    empty="none",
    clear="mouseout",
)

tooltips = alt.Chart(cat_index_dict['category_1'].reset_index()).transform_pivot(
    "category_1", "index", groupby=["date"]
).mark_rule(strokeWidth=2, color="grey").encode(
    x='date:T',
    opacity=alt.condition(hover, alt.value(0.75), alt.value(0)),
    tooltip=["date:T"]+[alt.Tooltip(f"{cat_1}:Q", format='.2f') for cat_1 in cat_index_dict['category_1']['category_1'].unique()],
).add_selection(hover)


(base + tooltips).configure_title(
    fontSize=17
).configure_axis(
    labelFontSize=13,
    titleFontSize=15
).configure_legend(
titleFontSize=14,
labelFontSize=13
) 

alt.LayerChart(...)

There's a few things I'm seeing from this graph. First is that bakery, drinks, and food cupboard items seem to be plateauing where as fresh and frozen items don't seem to be slowing down. Second, the fresh and frozen items have the 

## Price Changes for Fresh Food Subcategories

In [34]:
# hide_input

primary_cat = 'fresh-food'
mask = cat_index_dict['category_2']['category_1'] == primary_cat

selection = alt.selection_multi(fields=['category_2'], bind='legend')

base = alt.Chart(cat_index_dict['category_2'][mask].reset_index()).mark_line().encode(
    y=alt.Y(shorthand='index', scale=alt.Scale(zero=False), axis=alt.Axis(title='Index')),
    x=alt.X(shorthand='date', axis=alt.Axis(title='Week Commencing Monday')),
    color=alt.Color('category_2', legend=alt.Legend(title=f"{' '.join(primary_cat.split('-')).title()} Subcategory")),
    strokeDash='category_2',
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
).add_selection(
    selection
).properties(
    title=f"Index of Tesco Food Subcategories for {' '.join(primary_cat.split('-')).title()} Category",
    width=900,
    height=500
)

hover = alt.selection_single(
    fields=["date"],
    nearest=True,
    on="mouseover",
    empty="none",
    clear="mouseout",
)

tooltips = alt.Chart(cat_index_dict['category_2'][mask].reset_index()).transform_pivot(
    "category_2", "index", groupby=["date"]
).mark_rule(strokeWidth=2, color="grey").encode(
    x='date:T',
    opacity=alt.condition(hover, alt.value(0.75), alt.value(0)),
    tooltip=["date:T"]+[alt.Tooltip(f"{cat_1}:Q", format='.2f') for cat_1 in cat_index_dict['category_2'][mask]['category_2'].unique()],
).add_selection(hover)


(base + tooltips).configure_title(
    fontSize=17
).configure_axis(
    labelFontSize=13,
    titleFontSize=15
).configure_legend(
titleFontSize=14,
labelFontSize=13
) 

alt.LayerChart(...)

In [42]:
# hide_input

primary_cat = 'fresh-food'
subcat_1 = 'fresh-meat-and-poultry'
mask = (cat_index_dict['category_3']['category_1'] == primary_cat) & (cat_index_dict['category_3']['category_2'] == subcat_1)

selection = alt.selection_multi(fields=['category_3'], bind='legend')

base = alt.Chart(cat_index_dict['category_3'][mask].reset_index()).mark_line().encode(
    y=alt.Y(shorthand='index', scale=alt.Scale(zero=False), axis=alt.Axis(title='Index')),
    x=alt.X(shorthand='date', axis=alt.Axis(title='Week Commencing Monday')),
    color=alt.Color('category_3', legend=alt.Legend(title=f"{' '.join(subcat_1.split('-')).title()} Subcategory")),
    strokeDash='category_3',
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
).add_selection(
    selection
).properties(
    title=f"Index of Tesco Food Subcategories for {' '.join(subcat_1.split('-')).title()} Subcategory",
    width=900,
    height=500
)

hover = alt.selection_single(
    fields=["date"],
    nearest=True,
    on="mouseover",
    empty="none",
    clear="mouseout",
)

tooltips = alt.Chart(cat_index_dict['category_3'][mask].reset_index()).transform_pivot(
    "category_3", "index", groupby=["date"]
).mark_rule(strokeWidth=2, color="grey").encode(
    x='date:T',
    opacity=alt.condition(hover, alt.value(0.75), alt.value(0)),
    tooltip=["date:T"]+[alt.Tooltip(f"{cat_1}:Q", format='.2f') for cat_1 in cat_index_dict['category_3'][mask]['category_3'].unique()],
).add_selection(hover)


(base + tooltips).configure_title(
    fontSize=17
).configure_axis(
    labelFontSize=13,
    titleFontSize=15
).configure_legend(
titleFontSize=14,
labelFontSize=13
) 

alt.LayerChart(...)

In [43]:
# hide_input

primary_cat = 'fresh-food'
subcat_1 = 'cheese'
mask = (cat_index_dict['category_3']['category_1'] == primary_cat) & (cat_index_dict['category_3']['category_2'] == subcat_1)

selection = alt.selection_multi(fields=['category_3'], bind='legend')

base = alt.Chart(cat_index_dict['category_3'][mask].reset_index()).mark_line().encode(
    y=alt.Y(shorthand='index', scale=alt.Scale(zero=False), axis=alt.Axis(title='Index')),
    x=alt.X(shorthand='date', axis=alt.Axis(title='Week Commencing Monday')),
    color=alt.Color('category_3', legend=alt.Legend(title=f"{' '.join(subcat_1.split('-')).title()} Subcategory")),
    strokeDash='category_3',
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
).add_selection(
    selection
).properties(
    title=f"Index of Tesco Food Subcategories for {' '.join(subcat_1.split('-')).title()} Subcategory",
    width=900,
    height=500
)

hover = alt.selection_single(
    fields=["date"],
    nearest=True,
    on="mouseover",
    empty="none",
    clear="mouseout",
)

tooltips = alt.Chart(cat_index_dict['category_3'][mask].reset_index()).transform_pivot(
    "category_3", "index", groupby=["date"]
).mark_rule(strokeWidth=2, color="grey").encode(
    x='date:T',
    opacity=alt.condition(hover, alt.value(0.75), alt.value(0)),
    tooltip=["date:T"]+[alt.Tooltip(f"{cat_1}:Q", format='.2f') for cat_1 in cat_index_dict['category_3'][mask]['category_3'].unique()],
).add_selection(hover)


(base + tooltips).configure_title(
    fontSize=17
).configure_axis(
    labelFontSize=13,
    titleFontSize=15
).configure_legend(
titleFontSize=14,
labelFontSize=13
) 

alt.LayerChart(...)

In [44]:
# hide_input

primary_cat = 'fresh-food'
subcat_1 = 'fresh-fruit'
mask = (cat_index_dict['category_3']['category_1'] == primary_cat) & (cat_index_dict['category_3']['category_2'] == subcat_1)

selection = alt.selection_multi(fields=['category_3'], bind='legend')

base = alt.Chart(cat_index_dict['category_3'][mask].reset_index()).mark_line().encode(
    y=alt.Y(shorthand='index', scale=alt.Scale(zero=False), axis=alt.Axis(title='Index')),
    x=alt.X(shorthand='date', axis=alt.Axis(title='Week Commencing Monday')),
    color=alt.Color('category_3', legend=alt.Legend(title=f"{' '.join(primary_cat.split('-')).title()} Subcategory")),
    strokeDash='category_3',
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
).add_selection(
    selection
).properties(
    title=f"Index of Tesco Food Subcategories for {' '.join(subcat_1.split('-')).title()} Subcategory",
    width=900,
    height=500
)

hover = alt.selection_single(
    fields=["date"],
    nearest=True,
    on="mouseover",
    empty="none",
    clear="mouseout",
)

tooltips = alt.Chart(cat_index_dict['category_3'][mask].reset_index()).transform_pivot(
    "category_3", "index", groupby=["date"]
).mark_rule(strokeWidth=2, color="grey").encode(
    x='date:T',
    opacity=alt.condition(hover, alt.value(0.75), alt.value(0)),
    tooltip=["date:T"]+[alt.Tooltip(f"{cat_1}:Q", format='.2f') for cat_1 in cat_index_dict['category_3'][mask]['category_3'].unique()],
).add_selection(hover)


(base + tooltips).configure_title(
    fontSize=17
).configure_axis(
    labelFontSize=13,
    titleFontSize=15
).configure_legend(
titleFontSize=14,
labelFontSize=13
) 

alt.LayerChart(...)

## Price Changes for Frozen Food Subcategories

In [36]:
# hide_input

primary_cat = 'frozen-food'
mask = cat_index_dict['category_2']['category_1'] == primary_cat

selection = alt.selection_multi(fields=['category_2'], bind='legend')

base = alt.Chart(cat_index_dict['category_2'][mask].reset_index()).mark_line().encode(
    y=alt.Y(shorthand='index', scale=alt.Scale(zero=False), axis=alt.Axis(title='Index')),
    x=alt.X(shorthand='date', axis=alt.Axis(title='Week Commencing Monday')),
    color=alt.Color('category_2', legend=alt.Legend(title=f"{' '.join(primary_cat.split('-')).title()} Subcategory")),
    strokeDash='category_2',
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
).add_selection(
    selection
).properties(
    title=f"Index of Tesco Food Subcategories for {' '.join(primary_cat.split('-')).title()} Category",
    width=900,
    height=500
)

hover = alt.selection_single(
    fields=["date"],
    nearest=True,
    on="mouseover",
    empty="none",
    clear="mouseout",
)

tooltips = alt.Chart(cat_index_dict['category_2'][mask].reset_index()).transform_pivot(
    "category_2", "index", groupby=["date"]
).mark_rule(strokeWidth=2, color="grey").encode(
    x='date:T',
    opacity=alt.condition(hover, alt.value(0.75), alt.value(0)),
    tooltip=["date:T"]+[alt.Tooltip(f"{cat_1}:Q", format='.2f') for cat_1 in cat_index_dict['category_2'][mask]['category_2'].unique()],
).add_selection(hover)


(base + tooltips).configure_title(
    fontSize=17
).configure_axis(
    labelFontSize=13,
    titleFontSize=15
).configure_legend(
titleFontSize=14,
labelFontSize=13
) 

alt.LayerChart(...)

## Price Changes for Food Cupboard Subcategories

In [37]:
# hide_input

primary_cat = 'food-cupboard'
mask = cat_index_dict['category_2']['category_1'] == primary_cat

selection = alt.selection_multi(fields=['category_2'], bind='legend')

base = alt.Chart(cat_index_dict['category_2'][mask].reset_index()).mark_line().encode(
    y=alt.Y(shorthand='index', scale=alt.Scale(zero=False), axis=alt.Axis(title='Index')),
    x=alt.X(shorthand='date', axis=alt.Axis(title='Week Commencing Monday')),
    color=alt.Color('category_2', legend=alt.Legend(title=f"{' '.join(primary_cat.split('-')).title()} Subcategory")),
    strokeDash='category_2',
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
).add_selection(
    selection
).properties(
    title=f"Index of Tesco Food Subcategories for {' '.join(primary_cat.split('-')).title()} Category",
    width=900,
    height=500
)

hover = alt.selection_single(
    fields=["date"],
    nearest=True,
    on="mouseover",
    empty="none",
    clear="mouseout",
)

tooltips = alt.Chart(cat_index_dict['category_2'][mask].reset_index()).transform_pivot(
    "category_2", "index", groupby=["date"]
).mark_rule(strokeWidth=2, color="grey").encode(
    x='date:T',
    opacity=alt.condition(hover, alt.value(0.75), alt.value(0)),
    tooltip=["date:T"]+[alt.Tooltip(f"{cat_1}:Q", format='.2f') for cat_1 in cat_index_dict['category_2'][mask]['category_2'].unique()],
).add_selection(hover)


(base + tooltips).configure_title(
    fontSize=17
).configure_axis(
    labelFontSize=13,
    titleFontSize=15
).configure_legend(
titleFontSize=14,
labelFontSize=13
) 

alt.LayerChart(...)

## Price Changes for Drinks Subcategories

In [ ]:
# hide_input

primary_cat = 'drinks'
mask = cat_index_dict['category_2']['category_1'] == primary_cat

selection = alt.selection_multi(fields=['category_2'], bind='legend')

base = alt.Chart(cat_index_dict['category_2'][mask].reset_index()).mark_line().encode(
    y=alt.Y(shorthand='index', scale=alt.Scale(zero=False), axis=alt.Axis(title='Index')),
    x=alt.X(shorthand='date', axis=alt.Axis(title='Week Commencing Monday')),
    color=alt.Color('category_2', legend=alt.Legend(title=f"{' '.join(primary_cat.split('-')).title()} Subcategory")),
    strokeDash='category_2',
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
).add_selection(
    selection
).properties(
    title=f"Index of Tesco Food Subcategories for {' '.join(primary_cat.split('-')).title()} Category",
    width=900,
    height=500
)

hover = alt.selection_single(
    fields=["date"],
    nearest=True,
    on="mouseover",
    empty="none",
    clear="mouseout",
)

tooltips = alt.Chart(cat_index_dict['category_2'][mask].reset_index()).transform_pivot(
    "category_2", "index", groupby=["date"]
).mark_rule(strokeWidth=2, color="grey").encode(
    x='date:T',
    opacity=alt.condition(hover, alt.value(0.75), alt.value(0)),
    tooltip=["date:T"]+[alt.Tooltip(f"{cat_1}:Q", format='.2f') for cat_1 in cat_index_dict['category_2'][mask]['category_2'].unique()],
).add_selection(hover)


(base + tooltips).configure_title(
    fontSize=17
).configure_axis(
    labelFontSize=13,
    titleFontSize=15
).configure_legend(
titleFontSize=14,
labelFontSize=13
) 

## Price Changes for Bakery Subcategories

In [38]:
# hide_input

primary_cat = 'bakery'
mask = cat_index_dict['category_2']['category_1'] == primary_cat

selection = alt.selection_multi(fields=['category_2'], bind='legend')

base = alt.Chart(cat_index_dict['category_2'][mask].reset_index()).mark_line().encode(
    y=alt.Y(shorthand='index', scale=alt.Scale(zero=False), axis=alt.Axis(title='Index')),
    x=alt.X(shorthand='date', axis=alt.Axis(title='Week Commencing Monday')),
    color=alt.Color('category_2', legend=alt.Legend(title=f"{' '.join(primary_cat.split('-')).title()} Subcategory")),
    strokeDash='category_2',
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
).add_selection(
    selection
).properties(
    title=f"Index of Tesco Food Subcategories for {' '.join(primary_cat.split('-')).title()} Category",
    width=900,
    height=500
)

hover = alt.selection_single(
    fields=["date"],
    nearest=True,
    on="mouseover",
    empty="none",
    clear="mouseout",
)

tooltips = alt.Chart(cat_index_dict['category_2'][mask].reset_index()).transform_pivot(
    "category_2", "index", groupby=["date"]
).mark_rule(strokeWidth=2, color="grey").encode(
    x='date:T',
    opacity=alt.condition(hover, alt.value(0.75), alt.value(0)),
    tooltip=["date:T"]+[alt.Tooltip(f"{cat_1}:Q", format='.2f') for cat_1 in cat_index_dict['category_2'][mask]['category_2'].unique()],
).add_selection(hover)


(base + tooltips).configure_title(
    fontSize=17
).configure_axis(
    labelFontSize=13,
    titleFontSize=15
).configure_legend(
titleFontSize=14,
labelFontSize=13
) 

alt.LayerChart(...)

In [34]:
px.imshow(
    cat_index_dict['category_2'].pivot_table(index='date', columns=['category_2'], values='index').corr(),
    labels=dict(x="Secondary Category", y="Secondary Category", color="Correlation"),
    color_continuous_scale='balance',
    color_continuous_midpoint=0,
    aspect="auto",
    height=1000,
    width=1200
)

# Wrapping Up

PUT THE CONCLUSION HERE